In [ ]:
# STEP 1: Import Libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import files
import seaborn as sns

# STEP 2: Upload Tumor and Normal Images
def upload_images(label):
    print(f"Upload {label} images:")
    uploaded = files.upload()
    return uploaded

tumor_images = upload_images("Tumor (Positive)")
normal_images = upload_images("Normal (Negative)")

# STEP 3: Load and Preprocess Images
def load_images(image_dict, label_value):
    data = []
    labels = []
    for filename in image_dict.keys():
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img = cv2.imread(filename)
            if img is not None:
                img = cv2.resize(img, (256, 256))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = img / 255.0
                data.append(img)
                labels.append(label_value)
    return data, labels

tumor_data, tumor_labels = load_images(tumor_images, 1)
normal_data, normal_labels = load_images(normal_images, 0)

# Combine and convert to arrays
all_data = np.array(tumor_data + normal_data)
all_labels = np.array(tumor_labels + normal_labels)

# STEP 4: Train-Test Split
x_train, x_test, y_train, y_test = train_test_split(
    all_data, all_labels, test_size=0.2, stratify=all_labels, random_state=42)

# STEP 5: Build CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# STEP 6: Train Model
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(x_train, y_train, epochs=50, batch_size=16,
                    validation_data=(x_test, y_test),
                    callbacks=[early_stop])

# STEP 7: Evaluate Model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"\nTest Accuracy: {accuracy*100:.2f}%")

# STEP 8: Classification Report
y_pred = (model.predict(x_test) > 0.5).astype("int32")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Normal", "Tumor"], yticklabels=["Normal", "Tumor"])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

# STEP 9: Training Curves
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.tight_layout()
plt.show()

# STEP 10: Save Model to Google Drive in "models" folder
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define path to save model
model_dir = "/content/drive/MyDrive/models"
os.makedirs(model_dir, exist_ok=True)

# Save the model
model_path = os.path.join(model_dir, "kidney_tumor_classifier.h5")
model.save(model_path)

print(f"Model saved to: {model_path}")
